In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')

from sagemaker.processing import NetworkConfig
from sagemaker.workflow.parameters import ParameterString
from sagemaker_rightline.model import Configuration
from sagemaker_rightline.validations import (
    StepImagesExist,
    StepKmsKeyIdAsExpected,
    PipelineParametersAsExpected,
    ContainerImage,
    StepNetworkConfigAsExpected,
    StepLambdaFunctionExists,
    StepRoleNameExists,
    StepRoleNameAsExpected,
)
from sagemaker_rightline.rules import Equals, Contains
# from tests.utils import create_image
# from tests.fixtures.pipeline import get_sagemaker_pipeline
# from tests.fixtures.image_details import IMAGE_1_URI, IMAGE_2_URI
from moto import mock_ecr
import boto3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'tests.fixtures'

zsh:1: no matches found: [dev]


## Fetching SM Pipeline

In [14]:
sm_pipeline = get_sagemaker_pipeline(script_path="../tests/fixtures/fake_processing_script.py")

NameError: name 'get_sagemaker_pipeline' is not defined

## Defining function that validates the SM Pipeline

Note: In order to run this example offline/without AWS access, we're mocking ECR and publishing two mock images to it.

In [3]:
@mock_ecr
@create_image(
    boto3.client("ecr"), [ContainerImage(uri=IMAGE_1_URI), ContainerImage(uri=IMAGE_2_URI)]
)
def validate_sm_pipeline(sm_pipeline: "sagemaker.workflow.pipeline.Pipeline") -> "pandas.DataFrame":
    validations = [
        StepImagesExist(),
        PipelineParametersAsExpected(
            parameters_expected=[
                ParameterString(
                    name="parameter-1",
                    default_value="some-value",
                ),
            ],
            rule=Contains(),
        ),
        StepKmsKeyIdAsExpected(
            kms_key_id_expected="some/kms-key-alias",
            step_name="sm_training_step_sklearn",  # optional: if not set, will check all steps
            rule=Equals(),
        ),
        StepNetworkConfigAsExpected(
            network_config_expected=NetworkConfig(
                enable_network_isolation=False,
                security_group_ids=["sg-1234567890"],
                subnets=["subnet-1234567890"],
            ),
            rule=Equals(negative=True),
        ),
        StepLambdaFunctionExists(),
        StepRoleNameExists(),
        StepRoleNameAsExpected(
            role_name_expected="some-role-name",
            step_name="sm_training_step_sklearn",  # optional: if not set, will check all steps
            rule=Equals(),
        ),
    ]
    cm = Configuration(
        validations=validations,
        sagemaker_pipeline=sm_pipeline,
    )
    return cm.run(return_df=True)

## Running the validation and printing the report

In [34]:
report = validate_sm_pipeline(sm_pipeline)

In [36]:
report

,validation_name,subject,success,message
0,StepImagesExist,['123456789012.dkr.ecr.eu-west-1.amazonaws.com...,True,Images ['123456789012.dkr.ecr.eu-west-1.amazon...
1,PipelineParametersAsExpected,"[ParameterString(name='parameter-1', parameter...",False,"[ParameterString(name='parameter-1', parameter..."
2,StepKmsKeyIdAsExpected,['some/kms-key-alias'],True,['some/kms-key-alias'] does equal ['some/kms-k...
3,StepNetworkConfigAsExpected,"[{'enable_network_isolation': False, 'security...",False,[None] does not equal [{'enable_network_isolat...
